<a href="https://colab.research.google.com/github/rodiwaa/learnings-pocs/blob/main/notebooks/adk_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# plan
# - create simple greetin_agent
# - create tool to print hello or hey
# - wrap in functool
# - attach to greeting_agent
# - print
# - stream

In [ ]:
! pip install google-adk google-generativeai

In [32]:
from google.adk.agents import Agent, ParallelAgent
from google.adk.tools import google_search
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from IPython.display import display, Markdown

In [25]:
from getpass import getpass
# from google.generativeai import genai
import google.generativeai as genai
import os

api_key = getpass("enter google api_key: ")

genai.configure(api_key=api_key)

os.environ["GOOGLE_API_KEY"] = api_key

print("google api key set")

enter google api_key: ··········
google api key set


In [ ]:
# agent

# session

# userid

# runner

# run

In [33]:
# --- Agent Definition ---

# agents
def create_weather_agent():
    """Create the friendly news fetch agent"""
    return Agent(
        name="weather",
        model="gemini-2.5-flash",
        description="Agent specialized in fetching day forecast of given place.",
        instruction="""
        You are the "Weather Forecaster" Agent 🚗 - a specialized AI assistant that creates forecast weather.

        Your Mission:
        Get weather forecast for given location. Present in friendly manner.

        Guidelines:
        1. **General Weather**: Focus on general weather conditions like hot, sunny, rainy.
        2. **Temperature**: Create morning, afternoon, and evening forecast for day.
        3. **AQI Pollution Info**: Find relevant info on pollution levels. Mention AQI levels with categories (Good, Safe, Warning, Critical).
        4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

        RETURN summary in MARKDOWN FORMAT with clear time blocks.
        """,
        tools=[google_search],
        output_key="weather"
    )

def create_news_agent():
    """Create the Spontaneous Day Trip Generator agent"""
    return Agent(
        name="day_trip_agent",
        model="gemini-2.5-flash",
        description="Agent specialized in finding news headlines for major categories.",
        instruction="""
        You are the "News" Generator 🚗 - a specialized AI assistant that brings important news for the day.

        Your Mission:
        Get top 2 headlines and new article URL for categories - World, Sports (Cricket), Business, Technology, Local (Mumbai) only.

        Guidelines:
        1. **Headlines and URL**: Make sure headlines summarise gist of news item. Provide clickable URL.
        2. **Categories**: Stick to given categories only - World, Sports (Cricket), Business, Technology, Local (Mumbai) only.
        3. **Avoid**: Avoid these topics - Politics, Bollywood.

        RETURN news in MARKDOWN FORMAT with clear healines and working URLs.
        """,
        tools=[google_search],
        output_key="news"
    )

weather_agent = create_weather_agent()
news_agent = create_news_agent()
print(f"🧞 Agent '{weather_agent.name}' is created and ready to bring you weather updates!")
print(f"🧞 Agent '{news_agent.name}' is created and ready to bring you news!")


def create_orchestrator_agent():
  """Root agent to run weather, news agents in parallel"""
  return ParallelAgent(
      name="orchestrator",
      model="gemini-2.5-flash",
      description="Orchestrator agent to run weather and news agents in parallel.",
      instruction="""
      Coordinate with sub agents to run them in parallel.
      Gather results from all subagents
      Prepare response neatly under sections and sub headings in MARKDOWN format.
      """,
      sub_agents=[
          "weather_agent", "news_agent"
      ]
  )

orchestrator_agent = create_orchestrator_agent()
print(f"🧞 Agent '{orchestrator_agent.name}' is created and ready!")


🧞 Agent 'weather' is created and ready to bring you weather updates!
🧞 Agent 'day_trip_agent' is created and ready to bring you news!


ValidationError: 4 validation errors for ParallelAgent
sub_agents.0
  Input should be a valid dictionary or instance of BaseAgent [type=model_type, input_value='weather_agent', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type
sub_agents.1
  Input should be a valid dictionary or instance of BaseAgent [type=model_type, input_value='news_agent', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/model_type
model
  Extra inputs are not permitted [type=extra_forbidden, input_value='gemini-2.5-flash', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden
instruction
  Extra inputs are not permitted [type=extra_forbidden, input_value='\n      Coordinate with ...ARKDOWN format.\n      ', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden

In [ ]:
# run both

In [27]:
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )
    final_response = ""
    try:
       async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):

         if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
      final_response = f"An error occurred: {e}"

    print("\n" + "-"*50)
    print("✅ Final Response:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

session_service = InMemorySessionService()
my_user_id = "rodi_adventurer_001"

In [28]:
### NOT RUNNING THESE AGENTS INDIVIDUALLY NOW. RUN ORCHESTRATOR AGENT INSTEAD ###
async def run_weather_agent():
    # Create a new, single-use session for this query
    weather_session = await session_service.create_session(
        app_name=weather_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Find me next 3 days weather forecast for Mumbai"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(weather_agent, query, weather_session, my_user_id)

await run_weather_agent()

🗣️ User Query: 'Find me next 3 days weather forecast for Mumbai'

🚀 Running query for agent: 'weather' in session: '4750f6c6-eeaf-4ec5-9299-f550e39e7d89'...

--------------------------------------------------
✅ Final Response:


Here's your friendly 3-day weather forecast for Mumbai! Get ready for mostly sunny days and warm temperatures, but keep an eye on the air quality.

---

### **Saturday, January 24, 2026**

*   **General Weather:** Expect a beautiful, sunny day with clear skies throughout. It's perfect for enjoying the outdoors!
*   **Temperature:**
    *   **Morning:** Starting cool and pleasant around 20-22°C (68-72°F).
    *   **Afternoon:** Warming up nicely to 25-27°C (77-81°F).
    *   **Evening:** A comfortable drop to around 22-24°C (72-75°F), making it one of the best nights for comfortable sleep.
*   **AQI Pollution Info:** The Air Quality Index (AQI) is anticipated to be around 146, falling into the **Poor** category. This level can cause irritation for sensitive groups, so it's advisable to monitor air quality if you're planning extended outdoor activities.

### **Sunday, January 25, 2026**

*   **General Weather:** Another lovely day with abundant sunshine and clear skies. It's expected to be a touch hotter than Saturday.
*   **Temperature:**
    *   **Morning:** Starting mild around 21-23°C (70-73°F).
    *   **Afternoon:** Temperatures will climb to about 26-27°C (79-81°F).
    *   **Evening:** Enjoy a pleasant evening as temperatures settle around 23-25°C (73-77°F).
*   **AQI Pollution Info:** The AQI is forecast to be around 152, continuing in the **Poor** category. Those with respiratory conditions should be aware, especially during peak traffic hours, as dust and emissions can accumulate.

### **Monday, January 26, 2026**

*   **General Weather:** The week kicks off with plenty of sunshine. There's a slight chance of partly cloudy skies and a 5% chance of rain at night, but mostly it will be dry. This day looks to be the warmest of the three.
*   **Temperature:**
    *   **Morning:** A mild start, with temperatures hovering around 22-24°C (72-75°F).
    *   **Afternoon:** The warmest day, with highs reaching 27-28°C (81-82°F).
    *   **Evening:** Temperatures will cool down to approximately 24-26°C (75-79°F).
*   **AQI Pollution Info:** The AQI is predicted to be around 153, remaining in the **Poor** category. Given the persistent "Poor" air quality, consider wearing a mask if you're in areas with heavy traffic or spending significant time outdoors.

--------------------------------------------------



In [29]:
### NOT RUNNING THESE AGENTS INDIVIDUALLY NOW. RUN ORCHESTRATOR AGENT INSTEAD ###

async def run_news_agent():
    # Create a new, single-use session for this query
    news_session = await session_service.create_session(
        app_name=news_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Find me top news headlines for the day"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(news_agent, query, news_session, my_user_id)

await run_news_agent()

🗣️ User Query: 'Find me top news headlines for the day'

🚀 Running query for agent: 'day_trip_agent' in session: '71b7b124-fa7a-4c92-89a7-9da2f456f9ca'...

--------------------------------------------------
✅ Final Response:


Here are the top news headlines for the day:

### World
1.  **Japan pauses restart of world's largest nuclear power plant one day after it went online.**
    URL: https://www.theguardian.com/world/2026/jan/23/japan-pauses-restart-of-worlds-largest-nuclear-power-plant-one-day-after-it-went-online
2.  **Ukraine-Russia-US talks start in Abu Dhabi.**
    URL: https://www.theguardian.com/world/live/2026/jan/23/ukraine-russia-us-talks-start-in-abu-dhabi-europe-live

### Sports (Cricket)
1.  **Farewell, Shami: India pacer is done playing unless BCCI takes Kohli-Rohit route.**
    URL: https://www.hindustantimes.com/cricket/farewell-shami-india-pacer-is-done-playing-unless-bcci-takes-kohli-rohit-route-101705915834898.html
2.  **Black Caps dealt blow as pacer ruled out of T20 World Cup.**
    URL: https://www.icc-cricket.com/news/3884841

### Business
1.  **Heathrow scraps 100ml limit on liquids in cabin bags after tech upgrade.**
    URL: https://www.theguardian.com/business/2026/jan/23/heathrow-scraps-100ml-limit-on-liquids-in-cabin-bags-after-tech-upgrade
2.  **Amazon plans thousands of layoffs as corporate workforce shrinks.**
    URL: https://www.foxbusiness.com/economy/amazon-cut-thousands-jobs-sweeping-corporate-layoffs

### Technology
1.  **AI PCs' Unmet Promise Dragging Down Adoption.**
    URL: https://www.technewsworld.com/story/ai-pcs-unmet-promise-dragging-down-adoption-179373.html
2.  **TikTok Deal Is Finally Done: Joint venture aims to avoid US ban.**
    URL: https://www.pcmag.com/news/tiktok-deal-is-finally-done-after-all-the-drama-does-it-even-matter

### Local (Mumbai)
1.  **10% water cut in island city, eastern suburbs from Jan 27 for 11 days.**
    URL: https://indianexpress.com/article/cities/mumbai/10-water-cut-in-island-city-eastern-suburbs-from-jan-27-for-11-days-9122396/
2.  **Flyover between Dindoshi Court and Film City to be opened for traffic by May-end.**
    URL: https://www.mid-day.com/mumbai/mumbai-news/article/goregaon-mulund-link-road-flyover-nears-completion-bmc-aims-to-open-it-by-may-31-23330694

--------------------------------------------------



In [ ]:
async def run_orchestrator_agent():
    # Create a new, single-use session for this query
    orchestrator_session = await session_service.create_session(
        app_name=orchestrator_agent.name,
        user_id=my_user_id
    )

    # Note the new budget constraint in the query!
    query = "Find me top news headlines and weather updates for the day"
    print(f"🗣️ User Query: '{query}'")

    await run_agent_query(news_agent, query, orchestrator_session, my_user_id)

await run_orchestrator_agent()